# A dead simple markdown transformer for code cells

Previous implementations used python_line_transforms to transform the code.  With that approach a lot of effort was required to make sure that any other transformers were executed in the proper order.

In this experiment, the markdown transformer is baked directly in the `input_transform_manager`.

    %reload_ext deathbeds.__Markdown_code_cells

In [1]:
    from CommonMark import Parser
    from CommonMark.render.renderer import Renderer
    from textwrap import indent, dedent
    from IPython.display import display, Markdown
    from IPython.core.inputsplitter import IPythonInputSplitter

* Create a custom CommonMark renderer for code only.  It maintains the lines and positions of the original source to give better error messages.

* The commonmark renderer only catches code cells

    > There is an interesting discussion to be had about the role of inline cells.

In [2]:
    class CodeRenderer(Renderer):
        def code_block(self, node, entering):
            while len(self.buf.splitlines()) <  node.sourcepos[0][0]-1:
                self.out("\n")
            self.out(indent(node.literal, ' '*(
                node.sourcepos[0][1])))
            
        def __call__(self, str):
            return self.render(Parser().parse(str))

    def render(str): return CodeRenderer().render(Parser().parse(str))

    

In [3]:
    class MarkdownSplitter(IPythonInputSplitter):
        def transform_cell(self, str):
            if str.split('\n')[0].strip():
                display(Markdown(str))
            return super().transform_cell(dedent(render(str)))

* Export the splitter as an IPython extension for reusability.

In [4]:
    def load_ipython_extension(ip=None):
        ip = ip or get_ipython()
        ip.input_transformer_manager = MarkdownSplitter(
            physical_line_transforms=ip.input_transformer_manager.physical_line_transforms,
            logical_line_transforms=ip.input_transformer_manager.logical_line_transforms,
            python_line_transforms=ip.input_transformer_manager.python_line_transforms,
        )

* Test the Markdown Splitter

In [5]:
    if __name__ == '__main__':
        %load_ext deathbeds.__String_Node_Transformer
        %load_ext deathbeds.__Markdown_code_cells
        # This doesn't work if the node transformer comes afterwards because of shell=True
        # I don't think importnb should have shell as a default import state.

# testing

        """http://nbviewer.jupyter.org""";

In [6]:
    
    if __name__ == '__main__': import disqus